## **<h3 align="center"> Deep Learning - Project </h3>**
# **<h3 align="center"> Phylum Arthropoda - Steven</h3>**
**Group 4 members:**<br>
Alexandra Pinto - 20211599@novaims.unl.pt - 20211599<br>
Steven Carlson - 20240554@novaims.unl.pt - 20240554<br>
Sven Goerdes - 20240503@novaims.unl.pt - 20240503<br>
Tim Straub - 20240505@novaims.unl.pt - 20240505<br>
Zofia Wojcik  - 20240654@novaims.unl.pt - 20240654<br>

# Table of Contents
* [1. Introduction](#intro)
* [2. Setup](#setup)
* [3. Data Loading](#dataloading)
* [4. Image Preprocessing](#imagepreprocessing)
* [5. Neural Networks Models](#nnmodels)



# 1. Introduction <a class="anchor" id="intro"></a>

In this second notebook, we will preprocess images from the **Arthropoda** phylum and develop a deep learning model to accurately classify them at the family level.

# 2. Setup <a class="anchor" id="setup"></a>
In this section, we will import the necessary libraries that will be used throughout the notebook. These libraries will help with data handling and image processing.

In [111]:
# Standard libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import zipfile
import seaborn as sns

# Libraries for image processing
from glob import glob
from PIL import Image


In [ ]:
#Libraries from Keras / TensorFlow
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.utils import image_dataset_from_directory
from keras import optimizers
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_hub as hub

#Import pre-trained models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess




# 3. Data Loading <a class="anchor" id="dataloading"></a>

Let's open the train and test for Arthropoda Phylum.

In [113]:
# Load the DataFrame from the CSV file
arthropoda_train = pd.read_csv("/home/sacar/DeepLearning2425/train_test_splits/arthropoda_train.csv")
arthropoda_train.head(3)

,eol_content_id,eol_page_id,kingdom,phylum,family,file_path
0,28260809,1065329,animalia,arthropoda,apidae,arthropoda_apidae/28260809_1065329_eol-full-si...
1,29945328,1077217,animalia,arthropoda,pseudophasmatidae,arthropoda_pseudophasmatidae/29945328_1077217_...
2,14644212,463474,animalia,arthropoda,formicidae,arthropoda_formicidae/14644212_463474_eol-full...


In [114]:
# Load the DataFrame from the CSV file
arthropoda_test = pd.read_csv("/home/sacar/DeepLearning2425/train_test_splits/arthropoda_test.csv")
arthropoda_test.head(3)

,eol_content_id,eol_page_id,kingdom,phylum,family,file_path
0,28408206,1065346,animalia,arthropoda,apidae,arthropoda_apidae/28408206_1065346_eol-full-si...
1,28253620,1065348,animalia,arthropoda,apidae,arthropoda_apidae/28253620_1065348_eol-full-si...
2,21847584,1065348,animalia,arthropoda,apidae,arthropoda_apidae/21847584_1065348_eol-full-si...


# 4. Image Preprocessing <a class="anchor" id="imagepreprocessing"></a>

In [115]:
#Define some stuff
num_classes = arthropoda_train['family'].nunique() #number of classes = number of families
batch_size = 64
input_shape = (224, 224, 3)
image_size = (224, 224)
value_range = (0.0, 1.0)
num_classes = 17  



In [ ]:
#Define preprocess and augmentation functions

#Function to preprocess the images
def process_image(file_path, label):
    image = tf.io.read_file(file_path) # Read the image file
    image = tf.image.decode_jpeg(image, channels=3) # Decode the JPEG image
    image = tf.image.resize(image, image_size) # Resize the image to the target size
    
    #CHANGE THIS LINE DEPENDING ON WHICH PRE-TRAINED MODEL IS BEING USED
    
    #image = resnet_preprocess(image)  # Apply ResNet50 preprocessing
    #image = mobilenet_preprocess(image)  # Apply MobileNetV2 preprocessing
    #image = efficientnet_preprocess(image)  # Apply EfficientNetB0 preprocessing
    image = densenet_preprocess(image)  # Apply DenseNet121 preprocessing
    #image = inception_preprocess(image)  # Apply InceptionV3 preprocessing
    #image = tf.cast(image, tf.float32) / 255.0  # Apply ConvNeXt
    return image, label


#Function to augment the images
def augment_image(image, label):

    #Randomly change brightness
    image = tf.image.random_brightness(image, max_delta=0.2)

    #Apply geometric augmentations
    image = geometric_augmentation_layers(image, training=True) # Apply geometric augmentations
    image = tf.clip_by_value(image, 0.0, 1.0)
    
    return image, label


# Geometric augmentations
geometric_augmentation_layers = tf.keras.Sequential(
    [
        # Randomly flip horizontally
        tf.keras.layers.RandomFlip("horizontal"),

        # Randomly rotate
        tf.keras.layers.RandomRotation(factor=0.12),

        # Random zoom
        tf.keras.layers.RandomZoom(height_factor=(-0.35, 0.35), # Corresponds to [0.8, 1.2] of original height
                                   width_factor=(-0.35, 0.35)), # Corresponds to [0.8, 1.2] of original width

        # Random shift
        tf.keras.layers.RandomTranslation(height_factor=0.20,
                                          width_factor=0.20),

        # Contrast
        tf.keras.layers.RandomContrast(factor=0.25),

    ],
    name="geometric_augmentations",
)


In [117]:
root_dir = "/home/sacar/DeepLearning2425/rare_species"

#Get the file paths and labels for the training and test set
arthropoda_train['full_path'] = arthropoda_train['file_path'].apply(lambda x: os.path.normpath(os.path.join(root_dir, x)))
arthropoda_test['full_path'] = arthropoda_test['file_path'].apply(lambda x: os.path.normpath(os.path.join(root_dir, x)))

file_paths_train = arthropoda_train['full_path'].tolist()
labels_train = arthropoda_train['family'].tolist()

file_paths_test = arthropoda_test['full_path'].tolist()
labels_test = arthropoda_test['family'].tolist()

#Map the labels to integers
label_names = sorted(set(labels_train)) # Get the unique labels
label_to_index = {name: i for i, name in enumerate(label_names)} # Create a mapping from labels to integers
labels_train = [label_to_index[label] for label in labels_train]
labels_test = [label_to_index[label] for label in labels_test]

print(file_paths_train[:5])
print(labels_train[:5])
print(file_paths_test[:5])
print(labels_test[:5])

['/home/sacar/DeepLearning2425/rare_species/arthropoda_apidae/28260809_1065329_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_pseudophasmatidae/29945328_1077217_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_formicidae/14644212_463474_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_pseudophasmatidae/29945335_1077217_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_papilionidae/21035374_130548_eol-full-size-copy.jpg']
[0, 13, 5, 13, 10]
['/home/sacar/DeepLearning2425/rare_species/arthropoda_apidae/28408206_1065346_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_apidae/28253620_1065348_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_apidae/21847584_1065348_eol-full-size-copy.jpg', '/home/sacar/DeepLearning2425/rare_species/arthropoda_formicidae/14681521_403723_eol-full-size-copy.jpg', '/home/sacar/DeepLe

In [118]:
#Create the tensorflow datasets

#Load data
data = tf.data.Dataset.from_tensor_slices((file_paths_train, labels_train))
data = data.shuffle(buffer_size=len(file_paths_train), reshuffle_each_iteration=False, seed=42) # Shuffle the dataset

#Create train/val
train_size = int(0.8 * len(file_paths_train)) #80% for training, 20% for validation
train = data.take(train_size) # Take the first 80% for training
val = data.skip(train_size) # Skip the first 80% for validation

#Training preprocess pipeline
train = train.map(process_image, num_parallel_calls=tf.data.AUTOTUNE) # Map the function to the dataset
train = train.cache() # Cache the dataset for faster access
train = train.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE) # Map the function to the dataset
train = train.shuffle(buffer_size=len(file_paths_train), reshuffle_each_iteration=True, seed=42).batch(8).prefetch(buffer_size=tf.data.AUTOTUNE) #shuffle and batch

#Validation preprocess pipeline
val = val.map(process_image, num_parallel_calls=tf.data.AUTOTUNE) # Map the function to the dataset
val = val.cache() # Cache the dataset for faster access
val = val.batch(8).prefetch(buffer_size=tf.data.AUTOTUNE) #batch


#Test preprocess pipeline
test = tf.data.Dataset.from_tensor_slices((file_paths_test, labels_test))
test = test.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
test = test.cache().batch(8).prefetch(tf.data.AUTOTUNE)

In [119]:
for image, label in train.take(3):
    print("Image shape:", image.numpy().shape)
    print("Label:", label.numpy())

for image, label in test.take(3):
    print("Image shape:", image.numpy().shape)
    print("Label:", label.numpy())

Image shape: (8, 224, 224, 3)
Label: [0 8 5 5 2 8 0 5]
Image shape: (8, 224, 224, 3)
Label: [4 0 5 5 5 6 0 1]
Image shape: (8, 224, 224, 3)
Label: [ 3  5  3 11  6 10  1  3]
Image shape: (8, 224, 224, 3)
Label: [ 0  0  0  5  9  0 16  5]
Image shape: (8, 224, 224, 3)
Label: [ 5  5  0 16  6  0 11  0]
Image shape: (8, 224, 224, 3)
Label: [ 0  5 13  5  5  8  5  0]


2025-04-10 11:40:58.789859: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


# 5. Neural Network Models <a class="anchor" id="nnmodels"></a>

In [120]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

Physical devices cannot be modified after being initialized


In [121]:
# Print GPU devices detected
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU(s) detected: {[gpu.name for gpu in gpus]}")
else:
    print("❌ No GPU detected by TensorFlow.")

✅ GPU(s) detected: ['/physical_device:GPU:0']


In [ ]:
#Set the model to run
is_resnet = 0
is_mobilenet = 0
is_efficientnet = 0
is_densenet = 1
is_inception = 0
is_convnext = 0

In [123]:
if is_resnet == 1:
    # 1. Load the ResNet50 base model (without top classifier)
    base_model = ResNet50(
        weights='imagenet',          # Use ImageNet pretrained weights
        include_top=False,           # Exclude the default final dense layer
        input_shape=(224, 224, 3)    # Adjust to match your input images
    )

    # 2. Freeze the base model so we only train the classifier head
    base_model.trainable = False

    # 3. Build custom classification head
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(num_classes, activation='softmax')(x)


    # 4. Create the full model
    model = models.Model(inputs=base_model.input, outputs=output)

    # 5. Compile the model
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-5),
        loss='sparse_categorical_crossentropy',  # ✅ Use sparse version for integer labels
        metrics=['accuracy']
    )


    # 6. Train the model
    history = model.fit(
        train,
        epochs=100,
        validation_data=val,
        callbacks=[EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)],
        verbose=1
    )

    # 7. Save the model
    model.save("resnet50_rare_species_model.h5")

    plt.figure(figsize=(12, 5))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()



In [ ]:
if is_mobilenet == 1:
    # Load base model without top layer
    base_model = MobileNetV2(
        input_shape=(224, 224, 3),  # Match your resized image shape
        include_top=False,         # Don't include the original classifier
        weights='imagenet'         # Use ImageNet-pretrained weights
    )

    # Freeze base model so we only train the new layers for now
    base_model.trainable = False

    # Build custom model
    model = models.Sequential([
        base_model,                                # Feature extractor
        layers.GlobalAveragePooling2D(),           # Pool over spatial dimensions
        layers.Dropout(0.3),                       # Regularization layer
        layers.Dense(128, activation='relu'),      # Fully connected layer
        layers.Dense(num_classes, activation='softmax')      # Output layer (5 mollusk families)
    ])

    # Compile model
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss='sparse_categorical_crossentropy',  # Use sparse version for integer labels
        metrics=['accuracy']
    )

    # Summary for your viewing pleasure
    model.summary()

    history = model.fit(
        train,
        validation_data=val,
        epochs=100,           # Start small, increase if needed
        verbose=1
    )
    # Execution time: 9m 58.2s

: 

In [ ]:
if is_efficientnet == 1:
    # Load EfficientNetB0 as base model
    base_model = EfficientNetB0(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )

    # Freeze base model
    base_model.trainable = False

    # Build classifier on top
    model = models.Sequential([
        base_model,                                # Feature extractor
        layers.GlobalAveragePooling2D(),           # Pool features
        layers.Dropout(0.3),                       # Regularization
        layers.Dense(128, activation='relu'),      # Fully connected layer
        layers.Dense(num_classes, activation='softmax')  # Output layer
    ])

    # Compile the model
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Print summary
    model.summary()

    # Train it
    history = model.fit(
        train,
        validation_data=val,
        epochs=100,         # Adjust as needed
        verbose=1
    )


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_11     │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 17)             │         2,193 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,215,732 (16.08 MB)

 Trainable params: 166,161 (649.07 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

Epoch 1/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 63s 436ms/step - accuracy: 0.2578 - loss: 2.6305 - val_accuracy: 0.0338 - val_loss: 3.1140
Epoch 2/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.3029 - loss: 2.4749 - val_accuracy: 0.0338 - val_loss: 3.1588
Epoch 3/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.2882 - loss: 2.4413 - val_accuracy: 0.0338 - val_loss: 3.1755
Epoch 4/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.3405 - loss: 2.4151 - val_accuracy: 0.0338 - val_loss: 3.1772
Epoch 5/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.3005 - loss: 2.4014 - val_accuracy: 0.0338 - val_loss: 3.1964
Epoch 6/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.3045 - loss: 2.4694 - val_accuracy: 0.0338 - val_loss: 3.2190
Epoch 7/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.2879 - loss: 2.4912 - val_accuracy: 0.0338 - val_loss: 3.2346
Epoch 8/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.3314 - loss: 2.4002 - val_accuracy: 

In [ ]:
if is_densenet == 1:
    base_model = DenseNet121(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )

    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    history = model.fit(
        train,
        validation_data=val,
        epochs=100,
        verbose=1
    )

In [ ]:
if is_inception == 1:
    base_model = InceptionV3(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )

    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    history = model.fit(
        train,
        validation_data=val,
        epochs=100,
        verbose=1
    )

In [ ]:
if is_convnext == 1:
    base_model = tf.keras.Sequential([
        hub.KerasLayer(
            "https://tfhub.dev/sayakpaul/convnext_tiny_1k_224/1",
            trainable=False,
            input_shape=(224, 224, 3)
        )
    ])

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    history = model.fit(
        train,
        validation_data=val,
        epochs=100,
        verbose=1